# AutoGraph and tf.Module

In the last notebook, we explained why we should not define tf.Variable in the function decorated by @tf.function. It leads to the case that a function depends on a variable defined outside the function. It's absolutely not a perfect encapsulation. 

Tensorflow gives the class (tf.Module) to solve this problem. In fact, most middle level and high level API of tensorflow or keras is based or inherited from this class. 

With the class encapsulation, we define all the variable when initiating an object and all the methods as the class functions. 

In [1]:
import tensorflow as tf
import numpy

A sample case without class

In [2]:
x = tf.Variable(1.0,dtype=tf.float32)

@tf.function(input_signature=[tf.TensorSpec(shape = [], dtype = tf.float32)])    
def add_print(a):
    x.assign_add(a)
    tf.print(x)
    return(x)

In [3]:
add_print(tf.constant(3.0))

4


<tf.Tensor: shape=(), dtype=float32, numpy=4.0>

In [4]:
class DemoModule(tf.Module):
    def __init__(self,init_value=tf.constant(0.0),name=None):
        super(DemoModule, self).__init__(name=name)
        with self.name_scope:
            self.x = tf.Variable(init_value,dtype = tf.float32,trainable=True)


    @tf.function(input_signature=[tf.TensorSpec(shape = [], dtype = tf.float32)])  
    def addprint(self,a):
        with self.name_scope:
            self.x.assign_add(a)
            tf.print(self.x)
            return(self.x)

In [5]:
demo = DemoModule(tf.constant(1.0))

In [6]:
demo.addprint(tf.constant(9.0))

10


<tf.Tensor: shape=(), dtype=float32, numpy=10.0>

In [7]:
demo.addprint(tf.constant(9.0))

19


<tf.Tensor: shape=(), dtype=float32, numpy=19.0>

In [8]:
demo.variables

(<tf.Variable 'demo_module/Variable:0' shape=() dtype=float32, numpy=19.0>,)

In [9]:
demo.trainable_variables

(<tf.Variable 'demo_module/Variable:0' shape=() dtype=float32, numpy=19.0>,)

## tf.Module and tf.keras.Model, tf.keras.layers

In [13]:
from tensorflow.keras import models, layers, losses, metrics

In [14]:
print(issubclass(tf.keras.Model,tf.Module))
print(issubclass(tf.keras.layers.Layer,tf.Module))
print(issubclass(tf.keras.Model,tf.keras.layers.Layer))

True
True
True


In [15]:
tf.keras.backend.clear_session() 

model = models.Sequential()

model.add(layers.Dense(4,input_shape = (10,)))
model.add(layers.Dense(2))
model.add(layers.Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 44        
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 10        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 3         
Total params: 57
Trainable params: 57
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.variables

[<tf.Variable 'dense/kernel:0' shape=(10, 4) dtype=float32, numpy=
 array([[ 0.2541564 ,  0.3698734 , -0.5149219 , -0.14899957],
        [ 0.16546988,  0.44600976, -0.19985312, -0.35474944],
        [ 0.02563095,  0.26392227, -0.15051055, -0.5804306 ],
        [ 0.5596365 , -0.04437417, -0.61885214,  0.31763136],
        [-0.24365494,  0.16548085, -0.27752557,  0.05865592],
        [ 0.10062689, -0.07726318, -0.0992164 , -0.48314637],
        [ 0.28692842, -0.40627277,  0.577701  ,  0.64228344],
        [-0.09060055,  0.61619854,  0.4908607 , -0.04072994],
        [ 0.55513823, -0.06995642, -0.5790647 ,  0.03832799],
        [ 0.25808918, -0.46498674, -0.25116464,  0.08177114]],
       dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>,
 <tf.Variable 'dense_1/kernel:0' shape=(4, 2) dtype=float32, numpy=
 array([[ 0.00117278,  0.06719899],
        [ 0.03960061, -0.961905  ],
        [-0.79858637, -0.61166334],
        [ 

In [17]:
model.layers[0].trainable = False 
model.trainable_variables

[<tf.Variable 'dense_1/kernel:0' shape=(4, 2) dtype=float32, numpy=
 array([[ 0.00117278,  0.06719899],
        [ 0.03960061, -0.961905  ],
        [-0.79858637, -0.61166334],
        [ 0.85896754, -0.63283753]], dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>,
 <tf.Variable 'dense_2/kernel:0' shape=(2, 1) dtype=float32, numpy=
 array([[ 0.2464006 ],
        [-0.97346777]], dtype=float32)>,
 <tf.Variable 'dense_2/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

In [18]:
model.submodules

(<tensorflow.python.keras.engine.input_layer.InputLayer at 0x1379a5e48>,
 <tensorflow.python.keras.layers.core.Dense at 0x137889da0>)

In [19]:
model.name

'sequential'

In [20]:
model.name_scope()

'sequential'